In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import scipy.optimize as opt
from lmfit import models
from lmfit import Model

/home/frederik/anaconda3/lib/python3.4/site-packages/IPython/html.py:14: ShimWarning: The `IPython.html` package has been deprecated. You should import from `notebook` instead. `IPython.html.widgets` has moved to `ipywidgets`.
  "`IPython.html.widgets` has moved to `ipywidgets`.", ShimWarning)


In [4]:
pc_henrik = '/home/arrow13/Git/FP/Roentgen/'
pc_frederik = '/home/frederik/Dokumente/FP/Michelson/'

In [5]:
var_pc = pc_frederik

In [6]:
cd $var_pc/Messdaten

/home/frederik/Dokumente/FP/Michelson/Messdaten


# Spannung in Abhängigkeit der Chopperfrequenz

In [7]:
data_chopper = np.genfromtxt('chopper.dat',delimiter='\t').T

In [8]:
def chopper(x,A,B):
    return np.exp(A*np.log(x)+B)

In [9]:
model_chopper = Model(chopper,prefix='chop')
guess_chopper = model_chopper.make_params()
guess_chopper['chopA'].set(-0.8)
guess_chopper['chopB'].set(5)
out_chopper = model_chopper.fit(data_chopper[1],guess_chopper,x=data_chopper[0],weights=1/(data_chopper[3]/2))

In [10]:
out_chopper.plot_fit()
plt.show()

In [11]:
print(out_chopper.fit_report())

[[Model]]
    Model(chopper, prefix='chop')
[[Fit Statistics]]
    # function evals   = 36
    # data points      = 11
    # variables        = 2
    chi-square         = 31.234
    reduced chi-square = 3.470
[[Variables]]
    chopB:   8.91330759 +/- 0.096268 (1.08%) (init= 5)
    chopA:  -1.03784582 +/- 0.034072 (3.28%) (init=-0.8)
[[Correlations]] (unreported correlations are <  0.100)
    C(chopB, chopA)              = -0.982 


In [12]:
plt.errorbar(data_chopper[0],data_chopper[1],xerr=data_chopper[2],yerr=data_chopper[3],fmt='. ',label="Messdaten")
plt.plot(np.arange(5,80,0.1),out_chopper.eval(x=np.arange(5,80,0.1)),label='Fit')
plt.xlabel('Chopperfrequenz / Hz', fontsize=30)
plt.ylabel('Spannung / mV', fontsize=30)
plt.xticks(fontsize=20)
plt.yticks(fontsize=20)
plt.legend(loc='best',fontsize=30)
plt.show()

In [13]:
model_chopper_lin = models.LinearModel()
guess_chopper_lin = model_chopper_lin.guess(np.log(data_chopper[1]),x=np.log(data_chopper[0]))
out_chopper_lin = model_chopper_lin.fit(np.log(data_chopper[1]),
                                        guess_chopper_lin,
                                        x=np.log(data_chopper[0]),
                                        weights=1/((np.log(data_chopper[1]+data_chopper[3])+np.log(data_chopper[1]-data_chopper[3]))/2))

In [14]:
out_chopper_lin.plot()
plt.show()

In [15]:
print(out_chopper_lin.fit_report())

[[Model]]
    Model(linear)
[[Fit Statistics]]
    # function evals   = 6
    # data points      = 11
    # variables        = 2
    chi-square         = 0.001
    reduced chi-square = 0.000
[[Variables]]
    intercept:   8.89078790 +/- 0.099787 (1.12%) (init= 8.877967)
    slope:      -1.02941996 +/- 0.027973 (2.72%) (init=-1.025502)
[[Correlations]] (unreported correlations are <  0.100)
    C(intercept, slope)          = -0.990 


In [16]:
plt.errorbar(data_chopper[0],data_chopper[1],xerr=data_chopper[2],yerr=data_chopper[3],fmt='. ',label="Messdaten")
plt.plot(np.arange(5,1000,0.1),out_chopper.eval(x=np.arange(5,1000,0.1)),label='Fit')
plt.xlabel('Chopperfrequenz / Hz', fontsize=30)
plt.ylabel('Spannung / mV', fontsize=30)
plt.xticks(fontsize=20)
plt.yticks(fontsize=20)
plt.xscale('log')
plt.yscale('log')
plt.xlim([0,100])
plt.legend(loc='best',fontsize=30)
plt.show()

# Laserinterferogramm

## Eichung


In [17]:
data_laser_eichung = np.genfromtxt('eichung_laser.dat',delimiter='\t').T

In [18]:
plt.plot(data_laser_eichung[1],data_laser_eichung[0],'. ')
plt.show()

In [19]:
model = models.LinearModel()
guess = model.guess(data_laser_eichung[1],x=data_laser_eichung[0],weigths=1/0.5)
out = model.fit(data_laser_eichung[1],guess,x=data_laser_eichung[0])

In [20]:
out.plot()
plt.legend(loc='best')
plt.show()

In [21]:
print(out.fit_report())

[[Model]]
    Model(linear)
[[Fit Statistics]]
    # function evals   = 3
    # data points      = 12
    # variables        = 2
    chi-square         = 0.000
    reduced chi-square = 0.000
[[Variables]]
    intercept:   4.02639147 +/- 0.000922 (0.02%) (init= 4.026391)
    slope:       5.0096e-05 +/- 5.05e-08 (0.10%) (init= 5.009634e-05)
[[Correlations]] (unreported correlations are <  0.100)
    C(intercept, slope)          = -0.894 


In [22]:
(np.square((out.best_fit-data_laser_eichung[1]).sum()/5))

8.535815543890981e-16

In [23]:
plt.plot(data_laser_eichung[0],data_laser_eichung[1],'. ',label="Messdaten")
plt.plot(data_laser_eichung[0],out.eval(x=data_laser_eichung[0]),label="linearer Fit")
plt.xlabel('s / mm', fontsize=30)
plt.ylabel('Spannung / V', fontsize=30)
plt.xticks(fontsize=20)
plt.yticks(fontsize=20)
plt.legend(loc='best',fontsize=30)
plt.show()

In [24]:
def to_s_laser(x):
    return ((x*5.3328e-05)+4.99608814)

## Interferogramm

In [25]:
data_laser = np.genfromtxt('laser.txt',delimiter='\t').T

In [26]:
plt.plot(to_s_laser(data_laser[0]),data_laser[1],label='Messdaten')
plt.xlabel('s / mm', fontsize=30)
plt.ylabel('Spannung / V', fontsize=30)
plt.xticks(fontsize=20)
plt.yticks(fontsize=20)
plt.xlim([5,6.62])
plt.legend(loc='best',fontsize=30)
plt.show()

In [27]:
data_laser[0].shape

(29967,)

#### Letztes Maximum

In [28]:
plt.plot(to_s_laser(data_laser[0][29967-200:29967]),data_laser[1][29967-200:29967])
plt.show()

In [29]:
m_l = to_s_laser(data_laser[0][np.argmax(data_laser[1][29967-200:29967])+29967-200])
print(m_l)

6.604087324


#### Erstes Maximum

In [30]:
plt.plot(to_s_laser(data_laser[0][0:150]),data_laser[1][0:150])
plt.show()

In [31]:
m_1 = to_s_laser(data_laser[0][np.argmax(data_laser[1][0:150])])
print(m_1)

5.00888686


In [32]:
s_1 = m_l - m_1
print(s_1)
liste_s = []
liste_s.append(s_1)

1.595200464


In [140]:
liste_k = []
k_1 = 2*s_1/(3.39*173)*1000
print(k_1)
liste_k.append(k_1)

5.44000703872


In [34]:
laser_lambda = []
laser_lambda.append((2*(m_l-m_1))/(5*173)*1000)
print((2*(m_l-m_1))/(5*173)*1000)

3.68832477225


#### Zweits Maximum

In [35]:
plt.plot(to_s_laser(data_laser[0][150:300]),data_laser[1][150:300])
plt.show()

In [36]:
m_2 = to_s_laser(data_laser[0][np.argmax(data_laser[1][150:300])])
print(m_2)
liste_s.append(m_l-m_2)
print(m_l-m_2)

5.01075334
1.593333984


In [141]:
k_2 = 2*(m_l-m_2)/(3.39*176)*1000
print(k_2)
liste_k.append(k_2)

5.34102300885


In [38]:
print((2*(m_l-m_2))/(5*172)*1000)
laser_lambda.append((2*(m_l-m_2))/(5*172)*1000)

3.70542786977


#### Drittes Maximum

In [39]:
plt.plot(to_s_laser(data_laser[0][300:450]),data_laser[1][300:450])
plt.show()

In [40]:
m_3 = to_s_laser(data_laser[0][np.argmax(data_laser[1][300:450])])
liste_s.append(m_l-m_3)
print(m_3)
print(m_l-m_3)

5.012726476
1.591360848


In [142]:
k_3 = 2*(m_l-m_3)/(3.39*175)*1000
print(k_3)
liste_k.append(k_3)

5.36489118584


In [42]:
print((2*(m_l-m_3))/(5*171)*1000)
laser_lambda.append((2*(m_l-m_3))/(5*171)*1000)

3.72248151579


#### Viertes Maximum

In [43]:
plt.plot(to_s_laser(data_laser[0][450:650]),data_laser[1][450:650])
plt.show()

In [44]:
m_4 = to_s_laser(data_laser[0][np.argmax(data_laser[1][450:650])])
liste_s.append(m_l-m_4)
print(m_4)
print(m_l-m_4)

5.015179564
1.58890776


In [143]:
k_4 = 2*(m_l-m_4)/(3.39*174)*1000
print(k_4)
liste_k.append(k_4)

5.38740636761


In [46]:
print((2*(m_l-m_4))/(5*170)*1000)
laser_lambda.append((2*(m_l-m_4))/(5*170)*1000)

3.73860649412


#### Fünftes Maximum

In [47]:
plt.plot(to_s_laser(data_laser[0][650:850]),data_laser[1][650:850])
plt.show()

In [48]:
m_5 = to_s_laser(data_laser[0][np.argmax(data_laser[1][650:850])+650])
print(m_5)
liste_s.append(m_l-m_5)
print(m_l-m_5)

5.049309484
1.55477784


In [144]:
k_5 = 2*(m_l-m_5)/(3.39*173)*1000
print(k_5)
liste_k.append(k_5)

5.30215642744


In [50]:
print((2*(m_l-m_5))/(5*170)*1000)
laser_lambda.append((2*(m_l-m_5))/(5*170)*1000)

3.6583008


#### Sechtes Maximum

In [51]:
plt.plot(to_s_laser(data_laser[0][850:1050]),data_laser[1][850:1050])
plt.show()

In [52]:
m_6 = to_s_laser(data_laser[0][np.argmax(data_laser[1][850:1050])+850])
print(m_6)
liste_s.append(m_l-m_6)
print(m_l-m_6)

5.058961852
1.545125472


In [145]:
k_6 = 2*(m_l-m_6)/(3.39*172)*1000
print(k_6)
liste_k.append(k_6)

5.29987470673


In [54]:
print((2*(m_l-m_6))/(5*170)*1000)
laser_lambda.append((2*(m_l-m_6))/(5*170)*1000)

3.63558934588


#### Siebtes Maximum

In [55]:
plt.plot(to_s_laser(data_laser[0][1050:1250]),data_laser[1][1050:1250])
plt.show()

In [56]:
m_7 = to_s_laser(data_laser[0][np.argmax(data_laser[1][1050:1250])+1050])
print(m_7)
liste_s.append(m_l-m_7)
print(m_l-m_7)

5.068934188
1.535153136


In [146]:
k_7 = 2*(m_l-m_7)/(3.39*172)*1000
print(k_7)
liste_k.append(k_7)

5.26566898539


In [58]:
print((2*(m_l-m_7))/(5*170)*1000)
laser_lambda.append((2*(m_l-m_7))/(5*170)*1000)

3.61212502588


#### Achtes Maximum

In [59]:
plt.plot(to_s_laser(data_laser[0][1050:1250]),data_laser[1][1250:1450])
plt.show()

In [60]:
m_8 = to_s_laser(data_laser[0][np.argmax(data_laser[1][1250:1450])+1250])
print(m_8)
liste_s.append(m_l-m_8)
print(m_l-m_8)

5.078426572
1.525660752


In [147]:
k_8 = 2*(m_l-m_8)/(3.39*172)*1000
print(k_8)
liste_k.append(k_8)

5.23310952871


In [62]:
print((2*(m_l-m_8))/(5*170)*1000)
laser_lambda.append((2*(m_l-m_8))/(5*170)*1000)

3.58979000471


#### Neuntes Maximum

In [63]:
plt.plot(to_s_laser(data_laser[0][1450:1650]),data_laser[1][1450:1650])
plt.show()

In [64]:
m_9 = to_s_laser(data_laser[0][np.argmax(data_laser[1][1450:1650])+1450])
print(m_9)
liste_s.append(m_l-m_9)
print(m_l-m_9)

5.088185596
1.515901728


In [148]:
k_9 = 2*(m_l-m_9)/(3.39*172)*1000
print(k_9)
liste_k.append(k_9)

5.19963548055


In [66]:
print((2*(m_l-m_9))/(5*170)*1000)
laser_lambda.append((2*(m_l-m_9))/(5*170)*1000)

3.56682759529


#### Zehntes Maximum

In [67]:
plt.plot(to_s_laser(data_laser[0][1650:1850]),data_laser[1][1650:1850])
plt.show()

In [68]:
m_10 = to_s_laser(data_laser[0][np.argmax(data_laser[1][1650:1850])+650])
print(m_10)
liste_s.append(m_l-m_10)
print(m_l-m_10)

5.04408334
1.560003984


In [149]:
k_10 = 2*(m_l-m_10)/(3.39*172)*1000
print(k_10)
liste_k.append(k_10)

5.3509089113


In [70]:
print((2*(m_l-m_10))/(5*170)*1000)
laser_lambda.append((2*(m_l-m_10))/(5*170)*1000)

3.67059760941


In [151]:
k_bar = np.array(liste_k).mean()

### Mittelwert Bestimmen und Korrekturfaktor bestimmen

In [71]:
np.array(laser_lambda).mean()

3.6588071033105365

In [72]:
k = 2*np.array(liste_s).mean()/(3.39*173)*1000
print(k)

5.32181559773


## Interferogramm

In [73]:
data_laser = np.genfromtxt('laser.txt',delimiter='\t').T

In [74]:
plt.plot(data_laser[0],data_laser[1])
plt.xlim([0,30400])
plt.show()

# Weißlichtpunkt

## Eichung

### Erste Eichung
Start bei 5.6 $\mu$m

In [175]:
data_weisslicht_eichung = np.genfromtxt('eichung_weisspunkt.dat',delimiter="\t").T

In [176]:
plt.plot(data_weisslicht_eichung[0],data_weisslicht_eichung[1])
plt.show()

In [177]:
model = models.LinearModel()
guess = model.guess(data_weisslicht_eichung[1],x=data_weisslicht_eichung[0],weigths=1/50)
out = model.fit(data_weisslicht_eichung[1],guess,x=data_weisslicht_eichung[0])

In [178]:
out.plot()
plt.legend(loc='best')
plt.show()

In [179]:
print(out.fit_report())

[[Model]]
    Model(linear)
[[Fit Statistics]]
    # function evals   = 3
    # data points      = 12
    # variables        = 2
    chi-square         = 0.002
    reduced chi-square = 0.000
[[Variables]]
    intercept:   4.99608814 +/- 0.030543 (0.61%) (init= 4.996088)
    slope:       5.3328e-05 +/- 2.11e-06 (3.95%) (init= 5.332766e-05)
[[Correlations]] (unreported correlations are <  0.100)
    C(intercept, slope)          = -0.992 


In [174]:
plt.plot(data_weisslicht_eichung[0],data_weisslicht_eichung[1],'. ',label="Messdaten")
plt.plot()
plt.xlabel('s / $\mu$m', fontsize=30)
plt.ylabel('Spannung / mV', fontsize=30)
plt.xticks(fontsize=20)
plt.yticks(fontsize=20)
plt.legend(loc='best',fontsize=30)
plt.show()

In [81]:
def to_s_weisspunkt_1(x):
    return ((x*5.3328e-05)+4.99608814)

## Zweite Eichung
Start bei 4.65

In [82]:
data_weisslicht_eichung_2 = np.genfromtxt('eichung_weisspunkt_2.dat',delimiter="\t").T

In [83]:
plt.plot(data_weisslicht_eichung_2[0],data_weisslicht_eichung_2[1])
plt.show()

In [84]:
model = models.LinearModel()
guess = model.guess(data_weisslicht_eichung_2[1],x=data_weisslicht_eichung_2[0],weigths=1/50)
out = model.fit(data_weisslicht_eichung_2[1],guess,x=data_weisslicht_eichung_2[0])

In [85]:
out.plot()
plt.legend(loc='best')
plt.show()

In [86]:
print(out.fit_report())

[[Model]]
    Model(linear)
[[Fit Statistics]]
    # function evals   = 3
    # data points      = 10
    # variables        = 2
    chi-square         = 0.000
    reduced chi-square = 0.000
[[Variables]]
    intercept:   4.55051215 +/- 0.000873 (0.02%) (init= 4.550512)
    slope:       4.9858e-05 +/- 1.89e-07 (0.38%) (init= 4.98577e-05)
[[Correlations]] (unreported correlations are <  0.100)
    C(intercept, slope)          = -0.961 


In [87]:
plt.plot(data_weisslicht_eichung_2[0],data_weisslicht_eichung_2[1],'. ',label="Messdaten")
plt.xlabel('s / $\mu$m', fontsize=30)
plt.ylabel('Spannung / mV', fontsize=30)
plt.xticks(fontsize=20)
plt.yticks(fontsize=20)
plt.legend(loc='best',fontsize=30)
plt.show()

In [88]:
def to_s_weisspunkt_2(x):
    return ((x*4.9858e-05)+4.55051215)

## Interfreogramm

### Erstes Interferogramm

In [89]:
data_weisslicht = np.genfromtxt('weisspunkt.txt',delimiter="\t").T

In [90]:
#bestimmen den Weißlichtpunkts
print(to_s_weisspunkt_1(data_weisslicht[0][np.argmax(data_weisslicht[1])]))

5.773557052


In [91]:
#Fehler auf die Postion (bestimmt mit Fitparametern)
np.sqrt(np.square(0.0305)+np.square(1.89e-07 * 5.773557052))

0.030500000019520041

In [92]:
plt.plot(data_weisslicht[0],data_weisslicht[1],label='Messdaten')
plt.show()

In [93]:
to_s_weisspunkt_1(data_weisslicht[0])

array([ 5.5988012 ,  5.59885452,  5.59890785, ...,  5.92234217,
        5.9223955 ,  5.92244883])

In [94]:
plt.plot(to_s_weisspunkt_1(data_weisslicht[0]),data_weisslicht[1],label="Messdaten")
plt.plot(np.arange(0,10)/np.arange(0,10)*5.7735570520000001,np.arange(0,10), label="Weißlichtpunkt")
plt.xlabel('s / $\mu$m', fontsize=30)
plt.ylabel('Spannung / V', fontsize=30)
plt.xticks(fontsize=20)
plt.yticks(fontsize=20)
plt.xlim([5.70,5.84])
plt.ylim([1.7,3])
plt.legend(loc='best',fontsize=30)
plt.show()

## Zweites Interferogramm

In [95]:
data_weisslicht_2 = np.genfromtxt('weisspunkt_2.txt',delimiter="\t").T

In [96]:
#bestimmen den Weißlichtpunkts
print(to_s_weisspunkt_2(data_weisslicht_2[0][np.argmax(data_weisslicht_2[1])]))

4.754531086


In [97]:
#Fehler auf die Postion (bestimmt mit Fitparametern)
np.sqrt(np.square(0.000873)+np.square(2.11e-6 * 4.754531086))

0.00087305763970067329

In [98]:
plt.plot(data_weisslicht_2[0],data_weisslicht_2[1],label='Messdaten')
plt.show()

In [99]:
to_s_weisspunkt_2(data_weisslicht_2[0])

array([ 4.55370306,  4.55375292,  4.55380278, ...,  4.85828558,
        4.85833544,  4.8583853 ])

In [100]:
plt.plot(to_s_weisspunkt_2(data_weisslicht_2[0]),data_weisslicht_2[1],label="Messdaten")
plt.plot(np.arange(0,10)/np.arange(0,10)*4.754531086,np.arange(0,10), label="Weißlichtpunkt")
plt.xlabel('s / $\mu$m', fontsize=30)
plt.ylabel('Spannung / V', fontsize=30)
plt.xticks(fontsize=20)
plt.yticks(fontsize=20)
plt.xlim([4.65,4.85])
plt.ylim([2.4,3.6])
plt.legend(loc='best',fontsize=30)
plt.show()

# Schmalbandfilter
## Erster Weißlichtpunkt

### Eichung

In [101]:
data_schmalband_eichung = np.genfromtxt('eichung_filter_4,7',delimiter='\t').T

In [102]:
plt.plot(data_schmalband_eichung[0],data_schmalband_eichung[1])
plt.show()

In [103]:
model = models.LinearModel()
guess = model.guess(data_schmalband_eichung[1],x=data_schmalband_eichung[0],weigths=1/50)
out = model.fit(data_schmalband_eichung[1],guess,x=data_schmalband_eichung[0])

In [104]:
out.plot()
plt.legend(loc='best')
plt.show()

In [105]:
print(out.fit_report())

[[Model]]
    Model(linear)
[[Fit Statistics]]
    # function evals   = 4
    # data points      = 12
    # variables        = 2
    chi-square         = 0.000
    reduced chi-square = 0.000
[[Variables]]
    intercept:   4.65400408 +/- 0.003450 (0.07%) (init= 4.654004)
    slope:       4.4946e-05 +/- 1.54e-06 (3.42%) (init= 4.494635e-05)
[[Correlations]] (unreported correlations are <  0.100)
    C(intercept, slope)          = -0.928 


In [106]:
plt.plot(data_weisslicht_eichung_2[0],data_weisslicht_eichung_2[1],'. ',label="Messdaten")
plt.xlabel('s / $\mu$m', fontsize=30)
plt.ylabel('Spannung / mV', fontsize=30)
plt.xticks(fontsize=20)
plt.yticks(fontsize=20)
plt.legend(loc='best',fontsize=30)
plt.show()

In [107]:
def to_s_filter_1(x):
    return ((x*4.4946e-05)+4.65400409)

## Interferogramm

In [108]:
data_schmalband = np.genfromtxt('filter_4,7.txt',delimiter='\t').T

In [109]:
plt.plot(to_s_filter_1(data_schmalband[0]),data_schmalband[1],'b-',label='Messdaten')
plt.plot(np.arange(0,1.5,0.1)/np.arange(0,1.5,0.1)*4.7145,np.arange(0,1.5,0.1),'g-')
plt.plot(np.arange(0.46,0.9,0.1)/np.arange(0.46,0.9,0.1)*4.784,np.arange(0.46,0.9,0.1),'y-')
plt.plot(np.arange(4.66,4.8,0.1),np.arange(4.66,4.8,0.1)/np.arange(4.66,4.8,0.1)*0.7,'r-')
plt.show()

In [110]:
4.7145-4.6865

0.02800000000000047

$x_1$ = 4.6865

$x_2$ = 4.7145

$\delta$s = 0.028

n = 9

In [111]:
def wellenlänge(x_1,x_2,n):
    return (2*abs(x_1-x_2))/(5*n)

In [112]:
wellenlänge(4.7145,4.6865,1)*100

1.1200000000000188

## Zweiter Weißlichtpunkt

### Eichung

In [113]:
f_trafo = np.fft.fft(data_schmalband[1])
print(f_trafo)

[ 3045.60495600 +0.j           -35.49776458-55.53128682j
   -23.41780624+31.42583686j ...,     3.26728976+30.15344325j
   -23.41780624-31.42583686j   -35.49776458+55.53128682j]


In [114]:
data_schmalband_eichung_2 = np.genfromtxt('eichung_filter_2',delimiter='\t').T

In [115]:
plt.plot(data_schmalband_eichung_2[0],data_schmalband_eichung_2[1])
plt.show()

In [116]:
model = models.LinearModel()
guess = model.guess(data_schmalband_eichung_2[1],x=data_schmalband_eichung_2[0],weigths=1/50)
out = model.fit(data_schmalband_eichung_2[1],guess,x=data_schmalband_eichung_2[0])

In [117]:
out.plot()
plt.legend(loc='best')
plt.show()

In [118]:
print(out.fit_report())

[[Model]]
    Model(linear)
[[Fit Statistics]]
    # function evals   = 3
    # data points      = 15
    # variables        = 2
    chi-square         = 0.000
    reduced chi-square = 0.000
[[Variables]]
    intercept:   5.70505772 +/- 0.000259 (0.00%) (init= 5.705058)
    slope:       4.9598e-05 +/- 8.60e-08 (0.17%) (init= 4.959827e-05)
[[Correlations]] (unreported correlations are <  0.100)
    C(intercept, slope)          = -0.864 


In [119]:
plt.plot(data_schmalband_eichung_2[0],data_schmalband_eichung_2[1],'. ',label="Messdaten")
plt.xlabel('s / $\mu$m', fontsize=30)
plt.ylabel('Spannung / mV', fontsize=30)
plt.xticks(fontsize=20)
plt.yticks(fontsize=20)
plt.legend(loc='best',fontsize=30)
plt.show()

In [120]:
def to_s_filter_2(x):
    return ((x*4.9598e-05)+5.70505772)

### Interferogramm

In [121]:
data_schmalband_2 = np.genfromtxt('filter_2.txt',delimiter='\t').T

In [122]:
plt.plot(to_s_filter_2(data_schmalband[0]),data_schmalband[1],'b-',label='Messdaten')
plt.plot(np.arange(0.4,1.06,0.0001)/np.arange(0.4,1.06,0.0001)*5.7881,np.arange(0.4,1.06,0.0001),'g-')
plt.plot(np.arange(0.13,1.27,0.0001)/np.arange(0.13,1.27,0.0001)*5.77,np.arange(0.13,1.27,0.0001),'y-',label ='Weißlichtpunkt')
plt.plot(np.arange(0.4,1.06,0.0001)/np.arange(0.4,1.06,0.0001)*5.7485,np.arange(0.4,1.06,0.0001),'g-',label="Peaks zur Bestimmung der Wellenlänge")
plt.plot(np.arange(5.6,6,0.0001),np.arange(5.6,6,0.0001)/np.arange(5.6,6,0.0001)*0.7,'r-')
plt.xlim([5.7,5.92])
plt.xlabel('s / mm',fontsize=30)
plt.ylabel('Spannung / V',fontsize=30)
plt.xticks(fontsize=20)
plt.yticks(fontsize=20)
plt.legend(loc='best',fontsize=30)
plt.show()

In [156]:
wellenlänge(5.7881,5.7485,k_bar)*1000

2.9783011783127038

$x_1$ = 5.7485

$x_2$ = 5.7881

$\delta$s = 0.0396

n = 5

In [139]:
(3.31-2.9117647058823586)*100

39.82352941176415

In [125]:
0.7-0.13

0.57

In [126]:
0.7-0.4

0.29999999999999993

In [127]:
1/np.e*0.57

0.20969128146772212

In [128]:
a = 1/0.57

In [129]:
m = (0.4-0.57)/(5.788-5.77)

In [130]:
b = 0.57+9.44*5.77

In [131]:
(0.2096-b)/m - 5.77

0.03544470588257553

In [132]:
para_b = 1.988
para_a = 0.35*100

In [133]:
nu_1 = para_b*100 - 2/para_a

In [134]:
nu_2 = para_b*100 + 2/para_a

In [135]:
(1/nu_1 - 1/nu_2)*1000

0.002891739868995588

In [136]:
def B(x,a,b):
    return ((a)/(2*np.sqrt(np.pi))) * np.exp(-(((a**2*(b-(x)))**2)/(4)))

In [137]:
x = np.arange(1.9,2.2,0.00001)
plt.plot(x,10*B(x,para_a,para_b),label="Spektrale Verteilung")
plt.xlabel('$\lambda$ / $\mu$m', fontsize=30)
plt.ylabel(r'B(1/$\nu$) / (a.u.)', fontsize=30)
plt.xlim([1.98,2])
plt.xticks(fontsize=)
plt.legend(loc='best',fontsize=30)
plt.show()

SyntaxError: invalid syntax (<ipython-input-137-3389d19435a7>, line 6)

# Schwebung

In [162]:
eichung_schwebung = np.genfromtxt('eichung_schwebung.dat',delimiter='\t').T

In [165]:
model = models.LinearModel()
guess = model.guess(eichung_schwebung[1],x=eichung_schwebung[0],weigths=1/50)
out = model.fit(eichung_schwebung[1],guess,x=eichung_schwebung[0])

In [166]:
print(out.fit_report())

[[Model]]
    Model(linear)
[[Fit Statistics]]
    # function evals   = 3
    # data points      = 12
    # variables        = 2
    chi-square         = 0.000
    reduced chi-square = 0.000
[[Variables]]
    intercept:   3.14858566 +/- 0.002625 (0.08%) (init= 3.148586)
    slope:       5.0098e-05 +/- 7.78e-08 (0.16%) (init= 5.009768e-05)
[[Correlations]] (unreported correlations are <  0.100)
    C(intercept, slope)          = -0.992 


## Interferogram

In [157]:
data_schwebung = np.genfromtxt('schwebung.txt',delimiter='\t').T

In [171]:
plt.plot((5.0098e-05*data_schwebung[0])+3.14858566,data_schwebung[1])
#plt.xlim(1.67,2.018)
plt.xticks(fontsize = 15)
plt.yticks(fontsize = 15)
plt.ylabel('Spannung/100mV', size = 20)
plt.xlabel(r'2-fache Verschiebung des Spiegels 2$\delta_l$/mm', size = 20)
plt.show()